In [10]:
import sys
#sys.path.append('/home/luca/GitRepositories/Brancher')
sys.path.append('/home/luca/GitRepositories/Brancher')

import numpy as np

In this tutorial we will explain how to integrate deep learning models into your Brancher pipline. 

## Bulding models with NN components

All pytorch functions can be used in Brancher. These functions need to be imported from the Brancher function module which converts all PyTorch functions acting on torch.Tensor to Brancher functions acting on brancher.Variable.

Let's get started by defining a stochastic convolutional network on the MNIST dataset. The first step is to import the dataset:

In [11]:
import torchvision

# Data
image_size = 28
num_classes = 10
train = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=None)
test = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=None)
dataset_size = len(train)
input_variable = np.reshape(train.train_data.numpy(), newshape=(dataset_size, 1, image_size, image_size))
output_labels = train.train_labels.numpy()

In a Brancher model, datasets are stored in empirical variables. These are random variables that sample minibatches from a dataset.
In a supervised problem we need two empirical variables, one for the input images and the other for the labels. However, these two variables need to be sampled jointly as each image should be associated to a particular label. In Brancher, we can implement by this coupling by greating an additional RandomIndices variable that is a parent of both images and labels:

In [12]:
from brancher.standard_variables import EmpiricalVariable as Empirical
from brancher.standard_variables import RandomIndices

# Data sampling model
minibatch_size = 7

minibatch_indices = RandomIndices(dataset_size=dataset_size, batch_size=minibatch_size, 
                                  name="indices", is_observed=True)

x = Empirical(input_variable, indices=minibatch_indices, 
              name="x", is_observed=True)

labels = Empirical(output_labels, indices=minibatch_indices, 
                   name="labels", is_observed=True)

For example, we can import the PyTorch conv2d function from brancher.functions to build a stochastic 2D convolutional layer with gaussian weights. We can do this by defining oth weights and the input image as Randomvariables and using the Brancher conv2d function.

In [13]:
from brancher import functions as BF

from brancher.standard_variables import NormalVariable as Normal
from brancher.standard_variables import BinomialVariable as Binomial

in_channels = 1
out_channels = 5
image_size = 28
#x = Normal(loc=np.zeros((in_channels, image_size, image_size)),
#           scale=1.,
#           name="x")
Wk = Normal(loc=np.zeros((out_channels, in_channels, 3, 3)),
            scale=np.ones((out_channels, in_channels, 3, 3)),
            name="Wk")
z = Normal(BF.conv2d(x, Wk, padding=1), 0.1, name="z")

Here the random input a is convolved with the random filters W. We can now run the forward pass by sampling from the model:

In [15]:
print(z.get_sample(num_samples)["z"][0].shape)
print(len(z.get_sample(num_samples)["z"]))

(7, 5, 28, 28)
6


Note that in each of these samples both the input and the weigths are sampled independently.

We can now add a linear layer to this result to get a shallow convolutional classifier. We can do this by

In [16]:
num_classes = 10
Wl = Normal(loc=np.zeros((num_classes, image_size*image_size*out_channels)),
            scale=np.ones((num_classes, image_size*image_size*out_channels)),
            name="Wl")
b = Normal(loc=np.zeros((num_classes, 1)),
           scale=np.ones((num_classes, 1)),
           name="b")
reshaped_z = BF.reshape(z, shape=(image_size*image_size*out_channels, 1))
k = Binomial(1, 
             logits=BF.linear(reshaped_z, Wl, b), 
             name="k")

Here we needed to reshape the variable z as we need to perform matrix multiplication in the linear layer. Note that in Brancher you never need to explicitly consider the batch dimension. Batch properties are part of the data, not of the model and Brancher handle them automatically!

In [18]:
print(k.get_sample(num_samples)["k"][0].shape)
print(len(k.get_sample(num_samples)["k"]))

(7, 10, 1)
6


## Bulding models using Brancher layers

## Using existing PyTorch models in a Brancher models